In [ ]:
state=None; mask=None; deterministic=True


obs = env.reset()
for i in range(env.envs[0].total_window -env.envs[0].lookback_window-1):
#     i += 1
#     print(i)
#     import pdb;pdb.set_trace()
#     action, _states = model.predict(obs, deterministic=False)
    
    observation = np.array(obs)
    vectorized_env = model._is_vectorized_observation(observation, model.observation_space)

    observation = observation.reshape((-1,) + model.observation_space.shape)
    with model.sess.as_default():
        
        
        
#         actions, _, _ = model.step_model.step(observation, deterministic=deterministic)
        
        q_values, actions_proba = model.step_model.sess.run([model.step_model.q_values, model.step_model.policy_proba], {model.step_model.obs_ph: obs})
        print(actions_proba)
        print(q_values)
        if deterministic:
            actions = np.argmax(q_values, axis=1)
        else:
            # Unefficient sampling
            # TODO: replace the loop
            # maybe with Gumbel-max trick ? (http://amid.fish/humble-gumbel)
            actions = np.zeros((len(obs),), dtype=np.int64)
            for action_idx in range(len(obs)):
                actions[action_idx] = np.random.choice(model.step_model.n_actions, p=actions_proba[action_idx])
        actions, _, _ = actions, q_values, None

    if not vectorized_env:
        actions = actions[0]
    
    action, _states = actions, None
    
    obs, rewards, done, info = env.step(action)

In [ ]:
df_obs, df_prices = df_obs_and_prices(num, asset)
df_prices = propagate_index(from_df=df_obs, to_df=df_prices_all)

env = DummyVecEnv([lambda: AequamEnv(df_obs.iloc[::n_filter], df_prices.iloc[::n_filter],\
                                     lookback_window = lookback_window, reward_type = reward_type,\
                                     transaction_smoothing = transaction_smoothing)])

if algo == 'dqn':   
    model = DQN(DQN_MlpPolicy, env, param_noise=True, verbose=1, tensorboard_log='tmp/')
elif algo == 'ppo2' :   
    model = PPO2(MlpPolicy, env, verbose=1, tensorboard_log='tmp/')
#     total_timesteps *= 100
else:
    print(1/0)
    
model.learn(total_timesteps=total_timesteps)

env.envs[0].play_last_episode(model)

env.envs[0].print_pdf_report('test')